# Workflow to Validate NISAR L2 Transient Displacement Requirements

**Original code authored by:** NISAR Science Team Members and Affiliates  

*May 13, 2022*

*NISAR Solid Earth Team*

<div class="alert alert-warning">
Both the initial setup (<b>Prep A</b> section) and download of the data (<b>Prep B</b> section) should be run at the start of the notebook. And all subsequent sections NEED to be run in order.
</div>

<hr/>

## Define CalVal Site 

In [1]:
# Choose a site and track direction
# Available transient displacement validation sites: 
#        CentralValleyA144 : Central Valley in California Sentinel-1 track 144  

site='CentralValleyA144'

## Table of Contents:
<a id='secular_TOC'></a>

<hr/>

[**Prep A. Environment Setup**](#transient_prep_a)

[**Prep B. Data Staging**](#transient_prep_b)

[**1. Generate Interferogram Stack**](#transient_gen_ifg)
- [1.1.  Crop Interferograms](#transient_crop_ifg)

[**2. Optional Corrections**](#transient_opt_correction)
- [2.1. Solid Earth Tides Correction](#transient_solid_earth)
- [2.2. Tropospheric Delay Correction](#transient_tropo_corr)
- [2.3. Topographic Residual Correction ](#transient_tropo_res_corr)

[**3. Make GNSS LOS Measurements**](#transient_gnss_los)
- [3.1. Find Collocated GNSS Stations](#transient_co_gnss)  
- [3.2. Make GNSS LOS Measurements](#transient_gnss_los2) 
- [3.3. Make GNSS and InSAR Relative Displacements](#transient_gnss_insar)

[**4. NISAR Validation Approach 1: GNSS-InSAR Direct Comparison**](#transient_validation1)
- [4.1. Pair up GNSS stations and make measurement residuals](#transient_pair1)
- [4.2. Validate the requirement based on binned measurement residuals](#transient_bin1)
- [4.3. Result visulazation](#transient_result1)
- [4.3. Conclusion](#transient_conclusion1)

[**5. NISAR Validation Approach 2: Noise Level Validation**](#transient_validation2)
- [5.1. Randomly sample pixels and pair them up](#transient_pair2)
- [5.2. Validate the requirement based on binned measurement residuals](#transient_bin2)
- [5.3. Result visulazation](#transient_result2)
- [5.3. Conclusion](#transient_conclusion2)

[**6. Appendix: GNSS Position Plots**](#transient_appendix)

<hr/>

<a id='transient_prep_a'></a>
## Prep A. Environment Setup
Setup your environment for processing data

In [2]:
# load packages
import copy
import glob
import math
import os
import random
import subprocess
from datetime import datetime as dt
from pathlib import Path

import numpy as np
import pandas as pd
import pyproj
from matplotlib import pyplot as plt
from mintpy import smallbaselineApp
from mintpy.objects import gps
from mintpy.utils import readfile, utils as ut

from solid_utils.sampling import load_geo, samp_pair

In [3]:
################# Set Directories ##########################################
print('\nCurrent directory:',os.getcwd())

if 'work_dir' not in locals():
    work_dir = Path.cwd()/'work'/'transient_ouputs'/site

print("Work directory:", work_dir)
work_dir.mkdir(parents=True, exist_ok=True)
# Change to Workdir   
os.chdir(work_dir)
       
gunw_dir = work_dir/'products'
gunw_dir.mkdir(parents=True, exist_ok=True)
print("   GUNW    dir:", gunw_dir) 
    
mintpy_dir = work_dir/'MintPy' 
mintpy_dir.mkdir(parents=True, exist_ok=True)
print("   MintPy  dir:", mintpy_dir)
############################################################################
### List of CalVal Sites:
'''
Set NISAR calval sites:
    CentralValleyA144  : Central Valley
    OklahomaA107       : Oklahoma
    PuertoRicoD98      : Puerto Rico (Earthquake M6.4 on 20200107) - Descending track 
    PuertoRicoA135     : Puerto Rico (Earthquake M6.4 on 20200107 & large aftershock on 20200703) - Ascending track
    RidgecrestD71      : Ridgecrest  (Earthquake M7.2 on 20190705) - Descending track
    RidgecrestA64      : Ridgecrest  (Earthquake M7.2 on 20190705) - Ascending track

ARIA & MintPy parameters:
    calval_location : name
    download_region : download box in S,N,W,E format
    analysis_region : analysis box in S,N,W,E format (must be within download_region)
    download_start_date : download start date as YYYMMDD  
    download_end_date   : download end date as YYYMMDD
    earthquakeDate :  arbitrary date for testing with the central_valley dataset
    sentinel_track : sentinel track to download
    gps_ref_site_name : Name of the GPS site for InSAR re-referencing
    tempBaseMax' : maximum number of days, 'don't use interferograms longer than this value 
    ifgExcludeList : default is not to exclude any interferograms
    maskWater' :  interior locations don't need to mask water
'''
sites = {
    ##########  CENTRAL VALLEY ##############
    'CentralValleyA144' : {'calval_location' : 'Central_Valley',
            'download_region' : '"36.18 36.26 -119.91 -119.77"', # download box in S,N,W,E format
            'analysis_region' : '"35.77 36.75 -120.61 -118.06"', # analysis box in S,N,W,E format (must be within download_region)
            'download_start_date' : '20180101',
            'download_end_date' : '20190101',
            'earthquakeDate' : '20180412',                       # arbitrary date for testing with the central_valley dataset
            'sentinel_track' : '144',
            'gps_ref_site_name' : 'CAWO',
            'tempBaseMax' : 'auto',
            'ifgExcludeList' : 'auto',
            'maskWater' : False},                       # reference site for this area
    ##########  OKLAHOMA ##############
    'OklahomaA107' : {'calval_location' : 'Oklahoma',
            'download_region' : '"31.7 37.4 -103.3 -93.5"',      # download box in S,N,W,E format
            'analysis_region' : '"35.25 36.5 -100.5 -98.5"',     # analysis box in S,N,W,E format (must be within download_region)
            'download_start_date' : '20210101',
            'download_end_date' : '20210801',
            'earthquakeDate' : '20210328',                       # arbitrary date for testing with the Oklahoma dataset
            'sentinel_track' : '107',
            'gps_ref_site_name' : 'OKCL',
            'tempBaseMax' : 'auto',
            'ifgExcludeList' : 'auto',
            'maskWater' : False},
    ##########  PUERTO RICO ##############
    'PuertoRicoD98' : {'calval_location' : 'PuertoRicoDesc',
            'download_region' : '"17.5 18.9 -67.5 -66.0"',       # download box in S,N,W,E format
            'analysis_region' : '"17.9 18.5 -67.3 -66.2"',       # analysis box in S,N,W,E format (must be within download_region)
            'download_start_date' : '20190701',
            'download_end_date' : '20200930',
            'earthquakeDate' : '20200107',                       # date of M6.4 quake
            'sentinel_track' : '98',                             # descending track
            'gps_ref_site_name' : 'PRLT',
            'tempBaseMax' : 24,                                  # don't use interferograms longer than 24 days
            'ifgExcludeList' : 'auto', 
            'maskWater' : True},                                 # need to mask ocean around Puerto Rico island
    'PuertoRicoA135' : {'calval_location' : 'PuertoRicoAsc',
             'download_region' : '"17.5 18.9 -67.5 -66.0"',      # download box in S,N,W,E format
             'analysis_region' : '"17.9 18.5 -67.3 -66.2"',      # analysis box in S,N,W,E format (must be within download_region)
             'download_start_date' : '20190701',
             'download_end_date' : '20200930',
             'earthquakeDate' : '20200107',                      # date of M6.4 quake
             'earthquakeDate2' : '20200703',                     # date of large aftershock
             'sentinel_track' : '135',                           # ascending track
             'gps_ref_site_name' : 'PRLT',
             'tempBaseMax' : 24,                                 # don't use interferograms longer than 24 days
             'ifgExcludeList' : 'auto',
             'maskWater' : True},                                # need to mask ocean around Puerto Rico island
    ##########  RIDGECREST ##############
    'RidgecrestD71': {'calval_location' : 'RidgecrestD71',
                      'download_region' : '"34.5 37.5 -119.0 -116.0"', # download box in S,N,W,E format
                      'analysis_region' : '"34.7 37.2 -118.9 -116.1"', # analysis box in S,N,W,E format (must be within download_region)
                      'download_start_date' : '20190601',
                      'download_end_date' : '20190831',
                      'earthquakeDate' : '20190705',                   # M7.2 quake date at Ridgecrest
                      'sentinel_track' : '71',
                      'gps_ref_site_name' : 'ISLK',
                      'tempBaseMax' : 'auto',
                      'ifgExcludeList' : 'auto',
                      'maskWater' : False},
    'RidgecrestA64': {'calval_location' : 'Ridgecrest',
                      'download_region' : '"34.5 37.5 -119.0 -116.0"', # download box in S,N,W,E format
                      'analysis_region' : '"34.7 37.2 -118.9 -116.1"', # analysis box in S,N,W,E format (must be within download_region)
                      'download_start_date' : '20190101',
                      'download_end_date' : '20191231',
                      'earthquakeDate' : '20190705',                   # M7.2 quake date at Ridgecrest
                      'sentinel_track' : '64',
                      'gps_ref_site_name' : 'ISLK',
                      'tempBaseMax' : 'auto',
                      'ifgExcludeList' : '[50,121,123,124,125,126]',   # list of bad ifgs to exclude from time-series analysis
                      'maskWater' : False}
}
transient_available_sites = ['CentralValleyA144']

if site not in transient_available_sites:
    msg = '\nSelected site not available! Please select one of the following sites:: \n{}'.format(transient_available_sites)
    raise Exception(msg)
else:
    print('\nSelected site: {}'.format(site))
    for key, value in sites[site].items():
        print('   '+ key, ' : ', value)


Current directory: /home/jovyan/ATBD_kangl/methods/transient
Work directory: /home/jovyan/ATBD_kangl/methods/transient/work/transient_ouputs/CentralValleyA144
   GUNW    dir: /home/jovyan/ATBD_kangl/methods/transient/work/transient_ouputs/CentralValleyA144/products
   MintPy  dir: /home/jovyan/ATBD_kangl/methods/transient/work/transient_ouputs/CentralValleyA144/MintPy

Selected site: CentralValleyA144
   calval_location  :  Central_Valley
   download_region  :  "36.18 36.26 -119.91 -119.77"
   analysis_region  :  "35.77 36.75 -120.61 -118.06"
   download_start_date  :  20180101
   download_end_date  :  20190101
   earthquakeDate  :  20180412
   sentinel_track  :  144
   gps_ref_site_name  :  CAWO
   tempBaseMax  :  auto
   ifgExcludeList  :  auto
   maskWater  :  False


<a id='transient_prep_b'></a>
## Prep B. Data Staging

In this initial processing step, all the necessary Level-2 unwrapped interferogram products over specified time range and 12 days time intervels are gathered by the ARIA-tool and organized by Mintpy.

In [4]:
# option to control the use of pre-staged data; [False/True]
Use_Staged_Data = True
     
######### DO NOT CHANGE LINES BELOW ########

if Use_Staged_Data:
     # Check if a stage file from S3 already exist, if not try and download it
    if len(glob.glob('MintPy/inputs/*.h5')) == 0:
        os.chdir(work_dir.parents[0])
        zip_name = site + '.zip'
        print('Downloading:',  Path.cwd()/zip_name)
        if not os.path.isfile(zip_name):
            try:
                command = "aws s3 cp --no-sign-request s3://asf-jupyter-data-west/NISAR_SE/" + site + '.zip ' + zip_name
                subprocess.run(command, shell=True, check = True)
            except:
                command = 'wget --no-check-certificate --no-proxy "http://asf-jupyter-data-west.s3.amazonaws.com/NISAR_SE/' + site + '.zip" -q --show-progress'
                print('\nDownloading staged data ... ')
                subprocess.run(command, stdout=None, stderr=subprocess.PIPE, shell=True)
            finally:
                if (work_dir.parents[0]/zip_name).is_file():
                    print('Finished downloading!')
                else:
                    raise RuntimeError('Failed downloading Staged data!!! Install aws or wget to proceed')
                
        command = 'unzip ' + str(work_dir.parents[0]/zip_name) +'; rm ' + str(work_dir.parents[0]/zip_name)
        process = subprocess.run(command, shell=True)
        os.chdir(work_dir)
    if not os.path.exists(work_dir):
        raise Exception("Staged data for site {} not sucessfully generated. Please delete site-name folder {} and rerun this cell".format(site, work_dir))
    print('Finish preparing staged data for MintPy!!')
    
else:
    ##################### 1. Download (Aria) Interferograms from ASF ################
    print('NEEDED To Download ARIA GUNWs: \n Link to create account : https://urs.earthdata.nasa.gov/')
    earthdata_user = input('Please type your Earthdata username:')
    earthdata_password = input('Please type your Earthdata password:')
    print('NEEDED To Download DEMs: \n Link to create account : https://portal.opentopography.org/login')
    print('API key location: My Account > myOpenTopo Authorizations and API Key > Request API key')
    opentopography_api_key = input('Please type your OpenTopo API key:')

    ######################## USE ARIA-TOOLS TO DOWNLOAD GUNW ########################
    '''
    REFERENCE: https://github.com/aria-tools/ARIA-tools
    '''
    aria_download = '''ariaDownload.py -b {bbox} -u {user} -p {password} -s {start}  -e {end} -t {track} -o Count'''

    ###############################################################################
    print('CalVal site {}'.format(site))
    print('  Searching for available GUNW products:\n')

    command = aria_download.format(bbox = sites[site]['download_region'],
                                   start = sites[site]['download_start_date'],
                                   end = sites[site]['download_end_date'],
                                   track = sites[site]['sentinel_track'],
                                   user = earthdata_user,
                                   password = earthdata_password)
      
    process = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text = True, shell = True)
    print(process.stdout)

    ############## Download GUNW ##################
    print("Start downloading GUNW files ...")
    process = subprocess.run(command.split(' -o')[0], stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, shell=True)
    # Missing progressbar
    print("Downloaded {} GUNW files in: {}\n".format(len([(x) for x in os.listdir(gunw_dir) if x.endswith('.nc')]), gunw_dir))

    ############## DO little CLEANING ###########
    data_to_clean = ["avg_rates.csv", "ASFDataDload0.py", "AvgDlSpeed.png", "error.log"]

    for i, file in enumerate(data_to_clean):
        print('Cleaning unnecessary data {} in {}'.format(file, gunw_dir))
        (gunw_dir/file).unlink(missing_ok=True)

    #Delete error log file from workdir
    print('Cleaning unnecessary data error.log in {}'.format(work_dir))
    (work_dir/"error.log").unlink(missing_ok=True)

Downloading: /home/jovyan/ATBD_kangl/methods/transient/work/transient_ouputs/CentralValleyA144.zip
download: s3://asf-jupyter-data-west/NISAR_SE/CentralValleyA144.zip to ./CentralValleyA144.zip
Finished downloading!
Archive:  /home/jovyan/ATBD_kangl/methods/transient/work/transient_ouputs/CentralValleyA144.zip
   creating: CentralValleyA144/.ipynb_checkpoints/
  inflating: CentralValleyA144/MintPy/Central_Valley.cfg  
  inflating: CentralValleyA144/MintPy/avgSpatialCoh.h5  
  inflating: CentralValleyA144/MintPy/gps_enu2los.csv  
  inflating: CentralValleyA144/MintPy/timeseries.h5  
   creating: CentralValleyA144/MintPy/GPS/
  inflating: CentralValleyA144/MintPy/GPS/P304.tenv3  
  inflating: CentralValleyA144/MintPy/GPS/HUNT.tenv3  
  inflating: CentralValleyA144/MintPy/GPS/RNCH.tenv3  
  inflating: CentralValleyA144/MintPy/GPS/P572.tenv3  
  inflating: CentralValleyA144/MintPy/GPS/RAPT.tenv3  
  inflating: CentralValleyA144/MintPy/GPS/P294.tenv3  
  inflating: CentralValleyA144/MintPy/

<a id='transient_gen_ifg'></a>
# Generate Interferogram Stack

We use the open-source ARIA-tools package to download processed L2 interferograms over selected cal/val regions from the Alaska Satellite Facility archive and to stitch/crop the frame-based NISAR GUNW products. ARIA-tools uses a phase-minimization approach in the product overlap region to stitch the unwrapped and ionospheric phase, a mosaicing approach for coherence and amplitude, and extracts the geometric information from the 3D data cubes through a mosaicking of the 3D datacubes and subsequent intersection with a DEM. ARIA has been used to pre-process NISAR beta products derived from Sentinel-1 which have revealed interseismic deformation and creep along the San Andreas Fault system, along with subsidence, landsliding, and other signals. 

We use MintPy to load interferograms and auxiliary metadata (i.e. look angle, longitude and latitude step size) into HDF5 format which could be easily loaded into Python for further analysis.

<a id='transient_crop_ifg'></a>
## Crop Interferograms

In [5]:
# Crop Interferograms to Analysis Region
if not Use_Staged_Data:
    ###########################################################################################################
    # Set up ARIA product and mask data with GSHHS water mask:
    '''
    REQUIRED: Acquire API key to access/download DEMs

    Follow instructions listed here to generate and access API key through OpenTopography:
    https://opentopography.org/blog/introducing-api-keys-access-opentopography-global-datasets.
    '''
    
    if not os.path.exists(work_dir/'stack'):
        os.system('echo "{api_key}" > ~/.topoapi; chmod 600 ~/.topoapi'.format(api_key = str(opentopography_api_key)))
        print('Preparing GUNWs for MintPY....')
        if sites[site]['maskWater']:
            command = 'ariaTSsetup.py -f "products/*.nc" -b ' + sites[site]['analysis_region'] + ' --mask Download  --croptounion' # slow
        else: # skip slow mask download when we don't need to mask water
            command = 'ariaTSsetup.py -f "products/*.nc" -b ' + sites[site]['analysis_region'] + ' --croptounion'

        ################################## CROP & PREPARE STACK ###################################################
        result = subprocess.run(command, stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT, text=True, shell=True)
    print('Finish preparing GUNWs for MintPy!!')

Set Up MintPy Configuration file:

In [6]:
config_file_content = '''
mintpy.load.processor = aria
mintpy.load.unwFile = ../stack/unwrapStack.vrt
mintpy.load.corFile = ../stack/cohStack.vrt
mintpy.load.connCompFile = ../stack/connCompStack.vrt
mintpy.load.demFile = ../DEM/SRTM_3arcsec.dem
mintpy.load.incAngleFile = ../incidenceAngle/*.vrt
mintpy.load.azAngleFile = ../azimuthAngle/*.vrt
mintpy.load.waterMaskFile = ../mask/watermask.msk
mintpy.reference.lalo = auto
mintpy.topographicResidual.pixelwiseGeometry = no
mintpy.troposphericDelay.method = no
mintpy.topographicResidual = no
'''
config_file = mintpy_dir/(site+'.cfg')
config_file.write_text(config_file_content)

518

Data reorganization by Mintpy. The output of this step is an "inputs" directory containing two HDF5 files:
- ifgramStack.h5: This file contains 6 dataset cubes (e.g. unwrapped phase, coherence, connected components etc.) and multiple metadata
- geometryGeo.h5: This file contains geometrical datasets (e.g., incidence/azimuth angle, masks, etc.)

In [7]:
smallbaselineApp.main(f'{config_file} --dostep load_data --dir {mintpy_dir}'.split())

MintPy version v1.3.3, date 2022-04-14
--RUN-at-2022-05-25 20:14:30.688429--
Current directory: /home/jovyan/ATBD_kangl/methods/transient/work/transient_ouputs/CentralValleyA144
Run routine processing with smallbaselineApp.py on steps: ['load_data']
Remaining steps: ['modify_network', 'reference_point', 'quick_overview', 'correct_unwrap_error', 'invert_network', 'correct_LOD', 'correct_SET', 'correct_troposphere', 'deramp', 'correct_topography', 'residual_RMS', 'reference_date', 'velocity', 'geocode', 'google_earth', 'hdfeos5']
--------------------------------------------------
Project name: CentralValleyA144
Go to work directory: /home/jovyan/ATBD_kangl/methods/transient/work/transient_ouputs/CentralValleyA144/MintPy
read custom template file: /home/jovyan/ATBD_kangl/methods/transient/work/transient_ouputs/CentralValleyA144/MintPy/CentralValleyA144.cfg
update default template based on input custom template
No new option value found, skip updating /home/jovyan/ATBD_kangl/methods/transi

/opt/conda/envs/NISAR_SE/lib/python3.8/site-packages/mintpy/load_data.py:757: UserWarning: prep_aria.py failed. Assuming its result exists and continue...
  warnings.warn('prep_aria.py failed. Assuming its result exists and continue...')


In [8]:
ifgs_file = mintpy_dir/'inputs/ifgramStack.h5'
geom_file = mintpy_dir/'inputs/geometryGeo.h5'

**NOTE:** If the interferogram has a resolution lower than 100 m, we need multi-look the interferogram phase values before calculating the empirical semivarigram.

Load the date of interferograms into Python:

In [9]:
ifgs_date = readfile.read(ifgs_file,datasetName='date')[0]

In [10]:
_ifgs_date = np.empty_like(ifgs_date,dtype=dt)
for i in range(ifgs_date.shape[0]):
    start_date = ifgs_date[i,0].decode()
    end_date = ifgs_date[i,1].decode()
    start_date = dt.strptime(start_date, "%Y%m%d")
    end_date = dt.strptime(end_date, "%Y%m%d")
    _ifgs_date[i] = [start_date,end_date]
    
ifgs_date = _ifgs_date
del _ifgs_date

Remove interferograms with time interval other than 12 days:

In [11]:
del_row_index = []
for i in range(ifgs_date.shape[0]):
    time_interval = (ifgs_date[i,1]-ifgs_date[i,0]).days
    if time_interval != 12:
        del_row_index.append(i)
while i<ifgs_date.shape[0]-1:
    if ifgs_date[i,1]==ifgs_date[i+1,0]:
        del_row_index.append(i+1)
        i = i+2
    else:
        i = i+1

In [12]:
ifgs_date = np.delete(ifgs_date,del_row_index,0)

Identify independent interferograms (i.e., selected inteferograms do NOT share common dates):

In [13]:
del_row_index = []
i = 0
while i<ifgs_date.shape[0]-1:
    if ifgs_date[i,1]==ifgs_date[i+1,0]:
        del_row_index.append(i+1)
        i = i+2
    else:
        i = i+1

In [14]:
ifgs_date = np.delete(ifgs_date,del_row_index,0)

Then the phase and coherence of selected interferograms, geometrical datasets, and attribution of them are loaded into numpy array:

In [15]:
unwrapPhaseName = ['unwrapPhase-'+i[0].strftime('%Y%m%d')+'_'+i[1].strftime('%Y%m%d') for i in ifgs_date]
coherenceName = ['coherence-'+i[0].strftime('%Y%m%d')+'_'+i[1].strftime('%Y%m%d') for i in ifgs_date]

In [16]:
ifgs_unw,atr = readfile.read(ifgs_file,datasetName=unwrapPhaseName)
insar_displacement = -ifgs_unw*float(atr['WAVELENGTH'])/(4*np.pi)*1000 # unit in mm

insar_coherence = readfile.read(ifgs_file,datasetName=coherenceName)[0]
del ifgs_unw

Change default missing phase values in interferograms from 0.0 to `np.nan`.

In [17]:
insar_displacement[insar_displacement==0.0] = np.nan

<a id='transient_opt_correction'></a>
# Optional interferograms correction

Phase distortions related to solid earth and ocean tidal effects as well as those due to temporal variations in the vertical stratification of the atmosphere can be mitigated using the approaches described below. At this point, it is expected that these corrections will not be needed to validate the mission requirements, but they may be used to produce the highest quality data products. Typically, these are applied to the estimated time series product rather than to the individual interferograms, since they are a function of the time of each radar acquisition.

<a id='transient_solid_earth'></a>
## Solid Earth Tides Correction
[MintPy provides functionality for this correction.]

<a id='transient_tropo_corr'></a>
## Tropospheric Delay Correction
Optional atmospheric correction utilizes the PyAPS (Jolivet et al., 2011, Jolivet and Agram, 2012) module within GIAnT (or eventually a merged replacement for GIAnT and MintPy). PyAPS is well documented, maintained and can be freely downloaded. PyAPS is included in GIAnT distribution). PyAPS currently includes support for ECMWF’s ERA-Interim, NOAA’s NARR and NASA’s MERRA weather models. A final selection of atmospheric models to be used for operational NISAR processing will be done during Phase C.

[T]ropospheric delay maps are produced from atmospheric data provided by Global Atmospheric Models. This method aims to correct differential atmospheric delay correlated with the topography in interferometric phase measurements. Global Atmospheric Models (hereafter GAMs)... provide estimates of the air temperature, the atmospheric pressure and the humidity as a function of elevation on a coarse resolution latitude/longitude grid. In PyAPS, we use this 3D distribution of atmospheric variables to determine the atmospheric phase delay on each pixel of each interferogram.

The absolute atmospheric delay is computed at each SAR acquisition date. For a pixel a_i at an elevation z at acquisition date i, the four surrounding grid points are selected and the delays for their respective elevations are computed. The resulting delay at the pixel a_i is then the bilinear interpolation between the delays at the four grid points. Finally, we combine the absolute delay maps of the InSAR partner images to produce the differential delay maps used to correct the interferograms.

[MintPy provides functionality for this correction.]

<a id='transient_tropo_res_corr'></a>
## Topographic Residual Correction 
[MintPy provides functionality for this correction.]

**NOTE:** Phase deramping is not appplied here.

**NOTE:** If the solid earth tides correction for interferogram is applied, it should also be applied for GNSS observation.

Preliminary summary: we have load all data we need for processing:
- `atr`: metadata, including incident angle, longitude and latitude step width, etc;
- `insar_displacement`: LOS measurement from InSAR;
- `insar_coherence`: coherence value of the interferograms:
- `ifgs_date`: list of date pairs of two SAR images that form a interferogram.

<a id='transient_gnss_los'></a>
# Make GNSS LOS Measurements

<a id='transient_co_gnss'></a>
## Find Collocated GNSS Stations
The project will have access to L2 position data for continuous GNSS stations in third-party networks such NSF’s Plate Boundary Observatory, the HVO network for Hawaii, GEONET-Japan, and GEONET-New Zealand, located in target regions for NISAR solid earth calval. Station data will be post-processed by one or more analysis centers, will be freely available, and will have latencies of several days to weeks, as is the case with positions currently produced by the NSF’s GAGE Facility and separately by the University of Nevada Reno. Networks will contain one or more areas of high-density station coverage (2~20 km nominal station spacing over 100 x 100 km or more) to support validation of L2 NISAR requirements at a wide range of length scales.

Get space and time range for searching GNSS station:

In [18]:
length, width = int(atr['LENGTH']), int(atr['WIDTH'])
lat_step = float(atr['Y_STEP'])
lon_step = float(atr['X_STEP'])
N = float(atr['Y_FIRST'])
W = float(atr['X_FIRST'])
S = N+lat_step*(length-1)
E = W+lon_step*(width-1)

In [19]:
start_date_gnss = ifgs_date[0,0]
end_date_gnss = ifgs_date[-1,-1]

Search for collocated GNSS stations:

In [20]:
os.chdir(mintpy_dir)
site_names, site_lats, site_lons = gps.search_gps(SNWE=(S,N,W,E),
                                                  start_date=start_date_gnss.strftime('%Y%m%d'),
                                                  end_date=end_date_gnss.strftime('%Y%m%d'))
os.chdir(work_dir)
site_names = [str(stn) for stn in site_names]
print("Initial list of {} stations used in analysis:".format(len(site_names)))
print(site_names)

Initial list of 54 stations used in analysis:
['BEPK', 'CACO', 'CAFP', 'CAHA', 'CAKC', 'CAND', 'CARH', 'CAWO', 'CRCN', 'DONO', 'GR8R', 'HELB', 'HOGS', 'HUNT', 'KENN', 'LAND', 'LEMA', 'LONP', 'LOWS', 'MASW', 'MIDA', 'MNMC', 'MULN', 'P056', 'P281', 'P282', 'P283', 'P292', 'P293', 'P294', 'P296', 'P297', 'P298', 'P300', 'P304', 'P465', 'P467', 'P531', 'P540', 'P546', 'P547', 'P566', 'P571', 'P572', 'P573', 'P789', 'P790', 'PKDB', 'POMM', 'RAPT', 'RNCH', 'TBLP', 'TRAN', 'WLHG']


<a id='transient_gnss_los2'></a>
## Make GNSS LOS Measurements

In this step, the 3-D GNSS observations are projected into LOS direction. The InSAR observations are averaged 3 by 3 near the station positions.

**NOTE:** the number of pixels used in calculating the averaged phase values at the GPS location depends on the resolution of input data.

Get daily position solutions for GNSS stations:

In [21]:
#os.chdir(mint_dir)
displacement = {}
gnss_time_series = {}
gnss_time_series_std = {}
bad_stn = {}  #stations to toss
pixel_radius = 1   #number of InSAR pixels to average for comparison with GNSS

for counter,stn in enumerate(site_names):
    gps_obj = gps.GPS(site = stn, data_dir = str(mintpy_dir/'GPS'))
    gps_obj.open()
        
    # count number of dates in time range
    gps_obj.read_displacement()
    dates = gps_obj.dates
    for i in range(insar_displacement.shape[0]):
        start_date = ifgs_date[i,0]
        end_date = ifgs_date[i,-1]
        
        range_days = (end_date - start_date).days
        gnss_count = np.histogram(dates, bins=[start_date,end_date])
        gnss_count = int(gnss_count[0])
        #print(gnss_count)

        # select GNSS stations based on data completeness, here we hope to select stations with data frequency of 1 day and no interruption
        if range_days == gnss_count-1:
        #if start_date in dates and end_date in dates:
            disp_gnss_time_series,disp_gnss_time_series_std,site_latlon = gps_obj.read_gps_los_displacement(str(mintpy_dir/'inputs/geometryGeo.h5'),
                                                                                                            start_date=start_date.strftime('%Y%m%d'),
                                                                                                            end_date=end_date.strftime('%Y%m%d'))[1:4]
            x_value = round((site_latlon[1] - W)/lon_step)
            y_value = round((site_latlon[0] - N)/lat_step)

            #displacement from insar observation in the gnss station, averaged
            #Caution: If you expand the radius parameter farther than the bounding grid it will break. 
            disp_insar = insar_displacement[i,
                                            y_value-pixel_radius:y_value+pixel_radius, 
                                            x_value-pixel_radius:x_value+pixel_radius]
            if np.isfinite(disp_insar).sum() == 0:
                break
            disp_insar = np.nanmean(disp_insar)

            disp_gnss_time_series = disp_gnss_time_series*1000 # convert unit from meter to mm
            disp_gnss_time_series_std = disp_gnss_time_series_std*1000
            gnss_time_series[(i,stn)] = disp_gnss_time_series
            gnss_time_series_std[(i,stn)] = disp_gnss_time_series_std
            displacement[(i,stn)] = list(site_latlon)
            disp_gnss = disp_gnss_time_series[-1] - disp_gnss_time_series[0]

            displacement[(i,stn)].append(disp_gnss)
            displacement[(i,stn)].append(disp_insar)
        else:
            try:
                bad_stn[i].append(stn)
            except:
                bad_stn[i] = [stn]
#os.chdir(cwd)

calculating station lat/lon
reading time and displacement in east/north/vertical direction
reading time and displacement in east/north/vertical direction
calculating station lat/lon
reading time and displacement in east/north/vertical direction
reading time and displacement in east/north/vertical direction
calculating station lat/lon
reading time and displacement in east/north/vertical direction
reading time and displacement in east/north/vertical direction
calculating station lat/lon
reading time and displacement in east/north/vertical direction
reading time and displacement in east/north/vertical direction
calculating station lat/lon
reading time and displacement in east/north/vertical direction
reading time and displacement in east/north/vertical direction
calculating station lat/lon
reading time and displacement in east/north/vertical direction
reading time and displacement in east/north/vertical direction
calculating station lat/lon
reading time and displacement in east/north/vert

Do some data structure transformation:

In [22]:
gnss_time_series = dict(sorted(gnss_time_series.items()))
gnss_time_series_std = dict(sorted(gnss_time_series_std.items()))
displacement = dict(sorted(displacement.items()))
bad_stn = dict(sorted(bad_stn.items()))

In [23]:
gnss_time_series = pd.DataFrame.from_dict(gnss_time_series)
gnss_time_series_std = pd.DataFrame.from_dict(gnss_time_series_std)

In [24]:
displacement = pd.DataFrame.from_dict(displacement,orient='index',
                                      columns=['lat','lon','gnss_disp','insar_disp'])
displacement.index = pd.MultiIndex.from_tuples(displacement.index,names=['ifg index','station'])

If there are less than 3 GNSS stations, don't conduct comparison:

In [25]:
drop_index = []
for i in displacement.index.get_level_values(0).unique():
    if len(displacement.loc[i]) < 3:
        drop_index.append(i)
displacement=displacement.drop(drop_index)
# ifgs_date after drop for approach 1
ifgs_date_ap1=np.delete(ifgs_date,drop_index,axis=0)

All data needed for approach 1:

In [26]:
displacement

lat         lon  gnss_disp  insar_disp
ifg index station                                              
0         BEPK     35.878387 -118.075742  -9.415634   30.533264
          CACO     36.176403 -120.362236  -4.467896   29.042988
          CAFP     36.423996 -120.101854  -8.374451   16.169525
          CAHA     36.323436 -119.621285  -4.899353   20.533506
          CAKC     36.007974 -119.969504  -6.986755   25.565058
...                      ...         ...        ...         ...
13        P790     35.929147 -120.514393  -0.927216   -7.762661
          POMM     35.919909 -120.479809  -3.019073   -8.656868
          RNCH     35.899992 -120.523239   1.165283    4.563308
          TBLP     35.917409 -120.362868  12.203991    1.160611
          TRAN     36.650845 -120.245783   3.166565  -24.078894

[680 rows x 4 columns]

**NOTE:** 
- A more general critterion is needed for GNSS station selection. Here the stations with uninterrupted data are selected while, in Secular Requirement Validation, stations are selected by data completeness and standard variation.

<a id='transient_gnss_insar'></a>
## Make GNSS and InSAR Relative Displacements

Here we randomly select one reference site and make both the GNSS and InSAR measurements relative to that reference to remove a constant offset.

In [27]:
# reference GNSS stations to GNSS reference site
for i in displacement.index.get_level_values(0).unique():
    gps_ref_site_name = random.choice(displacement.loc[i].index.unique())
    displacement.loc[i,'gnss_disp'] = displacement.loc[i,'gnss_disp'].values - displacement.loc[(i,gps_ref_site_name),'gnss_disp']
    displacement.loc[i,'insar_disp'] = displacement.loc[i,'insar_disp'].values - displacement.loc[(i,gps_ref_site_name),'insar_disp']
    ref_x_value = round((displacement.loc[(i,gps_ref_site_name),'lon'] - W)/lon_step)
    ref_y_value = round((displacement.loc[(i,gps_ref_site_name),'lat'] - N)/lat_step)

    ref_disp_insar = insar_displacement[i,
                                        ref_y_value-pixel_radius:ref_y_value+1+pixel_radius, 
                                        ref_x_value-pixel_radius:ref_x_value+1+pixel_radius]
    ref_disp_insar = np.nanmean(ref_disp_insar)
    insar_displacement[i] -= ref_disp_insar

Plot GNSS stations on InSAR displacement fields

In [ ]:
cmap = copy.copy(plt.get_cmap('RdBu'))
#cmap.set_bad(color='black')
vmin, vmax = np.nanmin(insar_displacement), np.nanmax(insar_displacement)
for i in displacement.index.get_level_values(0).unique():
    fig, ax = plt.subplots()
    img1 = ax.imshow(insar_displacement[i], cmap=cmap,vmin=vmin,vmax=vmax, interpolation='nearest', extent=(W, E, S, N))
    ax.set_title(ifgs_date[i,0].strftime('%Y%m%d')+'-'+ifgs_date[i,1].strftime('%Y%m%d'))
    cbar1 = fig.colorbar(img1, ax=ax)
    cbar1.set_label('LOS displacement [mm]')

    for stn in displacement.loc[i].index:
        lon,lat = displacement.loc[(i,stn),'lon'],displacement.loc[(i,stn),'lat']
        color = cmap((displacement.loc[(i,stn),'gnss_disp']-vmin)/(vmax-vmin))
        ax.scatter(lon,lat,s=8**2,color=color,edgecolors='k')
        ax.annotate(stn,(lon,lat),color='black')

<a id='transient_validation1'></a>
# NISAR Validation Approach 1: GNSS-InSAR Direct Comparison

<a id='transient_pair1'></a>
## Pair up GNSS stations and make measurement residuals

We first pair up all GNSS stations and compare the relative measurement from both GNSS and INSAR. 

In [29]:
insar_disp = {}
gnss_disp = {}
ddiff_dist = {}
ddiff_disp = {}
abs_ddiff_disp = {}
for i in displacement.index.get_level_values(0).unique():
    displacement_i = displacement.loc[i]
    insar_disp_i = []
    gnss_disp_i = []
    ddiff_dist_i = []
    ddiff_disp_i = []

    for sta1 in displacement_i.index:
        for sta2 in displacement_i.index:
            if sta2 == sta1:
                break
            insar_disp_i.append(displacement_i.loc[sta1,'insar_disp']-displacement_i.loc[sta2,'insar_disp'])
            gnss_disp_i.append(displacement_i.loc[sta1,'gnss_disp']-displacement_i.loc[sta2,'gnss_disp'])
            ddiff_disp_i.append(gnss_disp_i[-1]-insar_disp_i[-1])
            g = pyproj.Geod(ellps="WGS84")
            _,_,distance = g.inv(displacement_i.loc[sta1,'lon'],displacement_i.loc[sta1,'lat'],
                                 displacement_i.loc[sta2,'lon'],displacement_i.loc[sta2,'lat'])
            distance = distance/1000 # convert unit from m to km
            ddiff_dist_i.append(distance)
    insar_disp[i]=np.array(insar_disp_i)
    gnss_disp[i]=np.array(gnss_disp_i)
    ddiff_dist[i]=np.array(ddiff_dist_i)
    ddiff_disp[i]=np.array(ddiff_disp_i)
    abs_ddiff_disp[i]=abs(np.array(ddiff_disp_i))

Plot to compare displacement from GNSS and InSAR:

In [ ]:
for i in displacement.index.get_level_values(0).unique():
    plt.figure(figsize=(11,7))
    disp_range = (min([*insar_disp[i],*gnss_disp[i]]),max([*insar_disp[i],*gnss_disp[i]]))
    plt.hist(insar_disp[i],bins=100,range=disp_range,color = "green",label='D_InSAR')
    plt.hist(gnss_disp[i],bins=100,range=disp_range,color="orange",label='D_GNSS', alpha=0.5)
    plt.legend(loc='upper right')
    plt.title(f"Displacements \n Date range {ifgs_date[i,0].strftime('%Y%m%d')}-{ifgs_date[i,1].strftime('%Y%m%d')} \n Number of station pairs used: {len(insar_disp[i])}")
    plt.xlabel('LOS Displacement (mm)')
    plt.ylabel('Number of Station Pairs')
    plt.show()

Plot Displacement Residuals Distribution:

In [ ]:
for i in displacement.index.get_level_values(0).unique():
    plt.figure(figsize=(11,7))
    plt.hist(ddiff_disp[i],bins = 100, color="darkblue",linewidth=1,label='D_gnss - D_InSAR')
    plt.legend(loc='upper right')
    plt.title(f"Residuals \n Date range {ifgs_date[i,0].strftime('%Y%m%d')}-{ifgs_date[i,1].strftime('%Y%m%d')} \n Number of stations pairs used: {len(ddiff_disp[i])}")
    plt.xlabel('Displacement Residual (mm)')
    plt.ylabel('N Stations')
    plt.show()

Plot Absolute Displacement Residuals As a Function of Distance:

In [ ]:
for i in displacement.index.get_level_values(0).unique():
    dist_th = np.linspace(min(ddiff_dist[i]),max(ddiff_dist[i]),100)
    acpt_error = 3*(1+np.sqrt(dist_th))
    plt.figure(figsize=(11,7))
    plt.scatter(ddiff_dist[i],abs_ddiff_disp[i],s=1)
    plt.plot(dist_th, acpt_error, 'r')
    plt.xlabel("Distance (km)")
    plt.ylabel("Amplitude of Displacement Residuals (mm)")
    plt.title(f"Residuals \n Date range {ifgs_date[i,0].strftime('%Y%m%d')}-{ifgs_date[i,1].strftime('%Y%m%d')} \n Number of stations pairs used: {len(ddiff_dist[i])}")
    plt.legend(["Mission Reqiurement","Measuement"])
    #plt.xlim(0,5)
    plt.show()

In [33]:
ddiff_dist_ap1 = list(ddiff_dist.values())
abs_ddiff_disp_ap1 = list(abs_ddiff_disp.values())

We have got all needed data for approach 1:
- `ddiff_dist_ap1`: distance of GNSS pairs,
- `abs_ddiff_disp_ap1`: absolute value of measurement redisuals,
- `ifgs_date_ap1`: list of date pairs of two SAR images that form a interferogram.

<a id='transient_bin1'></a>
## Validate the requirement based on binned measurement residuals

In [34]:
n_ifgs = len(ddiff_dist_ap1)

Bin all measurement residuals to check if they pass the requirements or not:

In [35]:
n_bins = 10
bins = np.linspace(0.1,50.0,num=n_bins+1)

In [36]:
n_all = np.empty([n_ifgs,n_bins+1],dtype=int) # number of points for each ifgs and bins
n_pass = np.empty([n_ifgs,n_bins+1],dtype=int) # number of points pass
for i in range(n_ifgs):
    inds = np.digitize(ddiff_dist_ap1[i],bins)
    for j in range(1,n_bins+1):
        rqmt = 3*(1+np.sqrt(ddiff_dist_ap1[i][inds==j]))# mission requirement for i-th ifgs and j-th bins
        rem = abs_ddiff_disp_ap1[i][inds==j] # relative measurement
        assert len(rqmt) == len(rem)
        n_all[i,j-1] = len(rem)
        n_pass[i,j-1] = np.count_nonzero(rem<rqmt)
    n_all[i,-1] = np.sum(n_all[i,0:-2])
    n_pass[i,-1] = np.sum(n_pass[i,0:-2])

In [37]:
ratio = n_pass/n_all
thresthod = 0.683 
#The assumed nature of Gaussian distribution gives a probability of 0.683 of being within one standard deviation.
success_or_fail = ratio>thresthod

<a id='transient_result1'></a>
## Result visualization

Convert the result to pandas DataFrame for better visulization:

In [38]:
def to_str(x:bool):
    if x==True:
        return 'true '
    elif x==False:
        return 'false '

success_or_fail_str = [list(map(to_str, x)) for x in success_or_fail]

columns = []
for i in range(n_bins):
    columns.append(f'{bins[i]:.2f}-{bins[i+1]:.2f}')
columns.append('total')

index = []
for i in range(len(ifgs_date_ap1)):
    index.append(ifgs_date_ap1[i,0].strftime('%Y%m%d')+'-'+ifgs_date_ap1[i,1].strftime('%Y%m%d'))

n_all_pd = pd.DataFrame(n_all,columns=columns,index=index)
n_pass_pd = pd.DataFrame(n_pass,columns=columns,index=index)
ratio_pd = pd.DataFrame(ratio,columns=columns,index=index)
success_or_fail_pd = pd.DataFrame(success_or_fail_str,columns=columns,index=index)

Number of data points in each bin:

In [39]:
n_all_pd

,0.10-5.09,5.09-10.08,10.08-15.07,15.07-20.06,20.06-25.05,25.05-30.04,30.04-35.03,35.03-40.02,40.02-45.01,45.01-50.00,total
20180103-20180115,24,66,69,65,55,35,33,35,30,32,412
20180127-20180208,26,74,77,73,60,37,40,42,41,37,470
20180220-20180304,26,74,76,73,60,37,39,41,40,37,466
20180316-20180328,26,74,76,73,60,37,39,41,40,37,466
20180409-20180421,26,73,77,72,60,35,40,35,34,27,452
20180503-20180515,26,72,74,62,54,32,32,32,33,34,417
20180527-20180608,27,73,78,72,60,35,41,36,35,27,457
20180620-20180702,25,66,74,67,57,36,39,43,41,36,448
20180807-20180819,23,56,70,59,56,33,35,39,33,34,404
20180831-20180912,23,56,70,61,56,33,38,39,34,35,410


Number of data points that below the curve:

In [40]:
n_pass_pd

,0.10-5.09,5.09-10.08,10.08-15.07,15.07-20.06,20.06-25.05,25.05-30.04,30.04-35.03,35.03-40.02,40.02-45.01,45.01-50.00,total
20180103-20180115,19,49,46,41,37,27,22,29,24,24,294
20180127-20180208,26,69,75,71,58,34,40,41,36,37,450
20180220-20180304,24,72,75,73,60,37,39,38,39,37,457
20180316-20180328,18,60,68,67,58,35,36,39,38,37,419
20180409-20180421,25,69,77,71,59,35,39,32,28,25,435
20180503-20180515,25,72,73,61,54,32,29,30,28,30,404
20180527-20180608,26,70,64,49,31,14,17,17,17,9,305
20180620-20180702,24,65,72,66,53,34,38,39,37,35,428
20180807-20180819,20,55,67,56,56,32,33,38,29,32,386
20180831-20180912,17,31,34,27,24,19,23,22,15,11,212


Percentage of pass:

In [41]:
s = ratio_pd.style
s.set_table_styles([  # create internal CSS classes
    {'selector': '.true', 'props': 'background-color: #e6ffe6;'},
    {'selector': '.false', 'props': 'background-color: #ffe6e6;'},
], overwrite=False)
s.set_td_classes(success_or_fail_pd)

,0.10-5.09,5.09-10.08,10.08-15.07,15.07-20.06,20.06-25.05,25.05-30.04,30.04-35.03,35.03-40.02,40.02-45.01,45.01-50.00,total
20180103-20180115,0.791667,0.742424,0.666667,0.630769,0.672727,0.771429,0.666667,0.828571,0.800000,0.750000,0.713592
20180127-20180208,1.000000,0.932432,0.974026,0.972603,0.966667,0.918919,1.000000,0.976190,0.878049,1.000000,0.957447
20180220-20180304,0.923077,0.972973,0.986842,1.000000,1.000000,1.000000,1.000000,0.926829,0.975000,1.000000,0.980687
20180316-20180328,0.692308,0.810811,0.894737,0.917808,0.966667,0.945946,0.923077,0.951220,0.950000,1.000000,0.899142
20180409-20180421,0.961538,0.945205,1.000000,0.986111,0.983333,1.000000,0.975000,0.914286,0.823529,0.925926,0.962389
20180503-20180515,0.961538,1.000000,0.986486,0.983871,1.000000,1.000000,0.906250,0.937500,0.848485,0.882353,0.968825
20180527-20180608,0.962963,0.958904,0.820513,0.680556,0.516667,0.400000,0.414634,0.472222,0.485714,0.333333,0.667396
20180620-20180702,0.960000,0.984848,0.972973,0.985075,0.929825,0.944444,0.974359,0.906977,0.902439,0.972222,0.955357
20180807-20180819,0.869565,0.982143,0.957143,0.949153,1.000000,0.969697,0.942857,0.974359,0.878788,0.941176,0.955446
20180831-20180912,0.739130,0.553571,0.485714,0.442623,0.428571,0.575758,0.605263,0.564103,0.441176,0.314286,0.517073


<a id='transient_conclusion1'></a>
## Conclusion

In [42]:
percentage = np.count_nonzero(ratio_pd['total']>thresthod)/n_ifgs
print(f"Percentage of interferograms passes the requirement: {percentage}")
if percentage >= 0.70:
    print('The interferogram stack passes the requirement.')
else:
    print('The interferogram stack fails the requirement.')

Percentage of interferograms passes the requirement: 0.7857142857142857
The interferogram stack passes the requirement.


<div class="alert alert-warning">
Approach 1 final result for CentralValleyA144: around 79% of interferograms passes the requirement.
</div>

<a id='transient_validation2'></a>
# NISAR Validation Approach 2: Noise Level Validation

In this validation (Approach #2), we evaluate the estimated secular deformation rate (Requirements 658) or co-seismic displacement (Requirement 660) from time series processing or the individual unwrapped interferogram (Requirement 663) over selected cal/val areas with negligible deformation. Any estimated deformation should thus be treated as noise and our goal is to evaluate the significance of this noise. In general, noise in the modeled displacement or the unwrapped interferogram is anisotropic, but here we neglect this anisotropy. Also, we assume the noise is stationary.

We first randomly sample measurements and pair up sampled pixel measurements. For each pixel-pair, the difference of their measurement becomes:
$$d\left(r\right)=|(f\left(x\right)-f\left(x-r\right))|$$
Estimates of $d(r)$ from all pairs are binned according to the distance r. In each bin, $d(r)$ is assumed to be a normal distribution.

**Note:** Now we simply assume there is no deformation in this study area and time interval. But in fact, it is hard to find a enough large area without any deformation. An more realistic solution is to apply a mask to mask out deformed regions.

In [43]:
n_ifgs = insar_displacement.shape[0]

Mask Pixels with Low Coherence (optional)

In [44]:
#insar_displacement[insar_coherence <0.6] = np.nan

Plot the coherence and InSAR measurements:

In [ ]:
cmap = plt.get_cmap('gray')

for i in range(n_ifgs):
    fig, ax = plt.subplots(figsize=[18, 5.5])
    img1 = ax.imshow(insar_coherence[i],cmap=cmap, interpolation='nearest',extent=(W, E, S, N))
    ax.set_title(f"Coherence \n Date range {ifgs_date[i,0].strftime('%Y%m%d')}-{ifgs_date[i,1].strftime('%Y%m%d')}")
    cbar1 = fig.colorbar(img1, ax=ax)
    cbar1.set_label('coherence')

In [ ]:
cmap = plt.get_cmap('RdBu')
for i in range(n_ifgs):
    fig, ax = plt.subplots(figsize=[18, 5.5])
    img1 = ax.imshow(insar_displacement[i], cmap=cmap, interpolation='nearest', extent=(W, E, S, N))
    ax.set_title(f"Interferogram \n Date range {ifgs_date[i,0].strftime('%Y%m%d')}-{ifgs_date[i,1].strftime('%Y%m%d')}")
    cbar1 = fig.colorbar(img1, ax=ax)
    cbar1.set_label('LOS displacement [mm]')

<a id='transient_pair2'></a>
## Randomly sample pixels and pair them up

Calculate the coordinate for every pixel:

In [47]:
X0,Y0 = load_geo(atr)
X0_2d,Y0_2d = np.meshgrid(X0,Y0)

For each interferogram, randomly selected pixels need to be paired up. In order to keep measurements independent, different pixel pairs can not share same pixel. This is achieved by pairing up in sequence, i.e., pairing up pixel number 1 and number 2, 3 and 4...

In [48]:
dist = []; rel_measure = []
for i in range(n_ifgs):
    dist_i, rel_measure_i = samp_pair(X0_2d,Y0_2d,insar_displacement[i],num_samples=1000000)
    dist.append(dist_i)
    rel_measure.append(rel_measure_i)

Show the statistical property of selected pixel pairs:

In [ ]:
for i in range(n_ifgs):
    fig, ax = plt.subplots(figsize=[18, 5.5])
    img1 = ax.hist(dist[i], bins=100)
    ax.set_title(f"Histogram of distance \n Date range {ifgs_date[i,0].strftime('%Y%m%d')}-{ifgs_date[i,1].strftime('%Y%m%d')}")
    ax.set_xlabel(r'Distance ($km$)')
    ax.set_ylabel('Frequency')
    ax.set_xlim(0,50)

In [ ]:
for i in range(n_ifgs):
    fig, ax = plt.subplots(figsize=[18, 5.5])
    img1 = ax.hist(rel_measure[i], bins=100)
    ax.set_title(f"Histogram of Relative Measurement \n Date range {ifgs_date[i,0].strftime('%Y%m%d')}-{ifgs_date[i,1].strftime('%Y%m%d')}")
    ax.set_xlabel(r'Relative Measurement ($mm$)')
    ax.set_ylabel('Frequency')

In [ ]:
dist_th = np.linspace(0,50,100)
rqmt = 3*(1+np.sqrt(dist_th))
for i in range(n_ifgs):
    fig, ax = plt.subplots(figsize=[18, 5.5])
    ax.plot(dist_th, rqmt, 'r')
    ax.scatter(dist[i], rel_measure[i], s=1, alpha=0.25)
    ax.set_title(f"Comparation between Relative Measurement and Requirement Curve \n Date range {ifgs_date[i,0].strftime('%Y%m%d')}-{ifgs_date[i,1].strftime('%Y%m%d')}")
    ax.set_ylabel(r'Relative Measurement ($mm$)')
    ax.set_xlabel('Distance (km)')
    ax.set_xlim(0,50)

We have got data used of approach 2:
- `dist`: distance of pixel pairs,
- `rel_measure`: relative measurement of pixel pairs,
- `ifgs_date`: list of date pairs of two SAR images that form a interferogram.

<a id='transient_bin2'></a>
## Validate the requirement based on binned measurement residuals

In [52]:
n_ifgs = len(dist)

Bin all measurement residuals to check if they pass the requirements or not:

In [53]:
n_bins = 10
bins = np.linspace(0.1,50.0,num=n_bins+1)

In [54]:
n_all = np.empty([n_ifgs,n_bins+1],dtype=int) # number of points for each ifgs and bins
n_pass = np.empty([n_ifgs,n_bins+1],dtype=int) # number of points pass
#ratio = np.empty([n_ifgs,n_bins+1]) # ratio
# the final column is the ratio as a whole
for i in range(n_ifgs):
    inds = np.digitize(dist[i],bins)
    for j in range(1,n_bins+1):
        rqmt = 3*(1+np.sqrt(dist[i][inds==j]))# mission requirement for i-th ifgs and j-th bins
        rem = rel_measure[i][inds==j] # relative measurement
        assert len(rqmt) == len(rem)
        n_all[i,j-1] = len(rem)
        n_pass[i,j-1] = np.count_nonzero(rem<rqmt)
    n_all[i,-1] = np.sum(n_all[i,0:-2])
    n_pass[i,-1] = np.sum(n_pass[i,0:-2])

In [55]:
ratio = n_pass/n_all
mean_ratio = np.array([np.mean(ratio[:,:-1],axis=1)])
ratio = np.hstack((ratio,mean_ratio.T))
thresthod = 0.683
#The assumed nature of Gaussian distribution gives a probability of 0.683 of being within one standard deviation.
success_or_fail = ratio>thresthod

<a id='transient_result2'></a>
## Result visualization

Convert the result to pandas DataFrame for better visulization:

In [56]:
def to_str(x:bool):
    if x==True:
        return 'true '
    elif x==False:
        return 'false '

success_or_fail_str = [list(map(to_str, x)) for x in success_or_fail]

columns = []
for i in range(n_bins):
    columns.append(f'{bins[i]:.2f}-{bins[i+1]:.2f}')
columns.append('total')

index = []
for i in range(len(ifgs_date)):
    index.append(ifgs_date[i,0].strftime('%Y%m%d')+'-'+ifgs_date[i,1].strftime('%Y%m%d'))

n_all_pd = pd.DataFrame(n_all,columns=columns,index=index)
n_pass_pd = pd.DataFrame(n_pass,columns=columns,index=index)
ratio_pd = pd.DataFrame(ratio,columns=columns+['mean'],index=index)
success_or_fail_pd = pd.DataFrame(success_or_fail_str,columns=columns+['mean'],index=index)

Number of data points in each bin:

In [57]:
n_all_pd

,0.10-5.09,5.09-10.08,10.08-15.07,15.07-20.06,20.06-25.05,25.05-30.04,30.04-35.03,35.03-40.02,40.02-45.01,45.01-50.00,total
20180103-20180115,1600,4561,6972,9352,11450,13374,15128,16390,17541,18464,96368
20180127-20180208,1514,4478,7089,9307,11562,13005,14891,16236,17321,18535,95403
20180220-20180304,1605,4386,6939,9349,11593,13353,14938,16317,17800,18572,96280
20180316-20180328,1599,4413,7008,9340,11633,13438,14894,16363,17691,18496,96379
20180409-20180421,1591,4437,6851,9543,11515,13473,14886,16257,17720,18378,96273
20180503-20180515,1563,4508,7069,9383,11642,13412,14925,16354,17270,18461,96126
20180527-20180608,1584,4569,6975,9467,11405,13603,15143,16133,17497,18626,96376
20180620-20180702,1610,4406,7034,9473,11515,13225,14989,16490,17400,18478,96142
20180807-20180819,1604,4422,7173,9392,11622,13344,15091,16432,17733,18493,96813
20180831-20180912,1616,4497,7122,9326,11540,13331,14962,16303,17461,18638,96158


Number of data points that below the curve:

In [58]:
n_pass_pd

,0.10-5.09,5.09-10.08,10.08-15.07,15.07-20.06,20.06-25.05,25.05-30.04,30.04-35.03,35.03-40.02,40.02-45.01,45.01-50.00,total
20180103-20180115,1551,4403,6761,9092,11139,12988,14695,15871,17018,17965,93518
20180127-20180208,1463,4270,6757,8809,10875,12109,13849,14850,15631,16538,88613
20180220-20180304,1401,3800,6090,8300,10358,11993,13381,14644,15889,16670,85856
20180316-20180328,1376,3846,6186,8237,10245,11847,13185,14287,15442,16021,84651
20180409-20180421,1509,4126,6253,8471,9990,11435,12351,13265,14193,14238,81593
20180503-20180515,1435,4179,6615,8860,11025,12748,14277,15665,16545,17706,91349
20180527-20180608,1428,4008,5921,7805,9046,10546,11348,11751,12526,12974,74379
20180620-20180702,1529,4086,6391,8572,10375,11823,13425,14861,15789,17009,86851
20180807-20180819,1398,3760,5928,7653,9247,10346,11696,12589,13366,13863,75983
20180831-20180912,1396,3751,5814,7483,9088,10209,11268,11878,12416,12896,73303


Ratio of pass:

In [59]:
s = ratio_pd.style
s.set_table_styles([  # create internal CSS classes
    {'selector': '.true', 'props': 'background-color: #e6ffe6;'},
    {'selector': '.false', 'props': 'background-color: #ffe6e6;'},
], overwrite=False)
s.set_td_classes(success_or_fail_pd)

,0.10-5.09,5.09-10.08,10.08-15.07,15.07-20.06,20.06-25.05,25.05-30.04,30.04-35.03,35.03-40.02,40.02-45.01,45.01-50.00,total,mean
20180103-20180115,0.969375,0.965358,0.969736,0.972198,0.972838,0.971138,0.971378,0.968334,0.970184,0.972974,0.970426,0.970351
20180127-20180208,0.966314,0.953551,0.953167,0.946492,0.940581,0.931103,0.930025,0.914634,0.902431,0.892258,0.928828,0.933056
20180220-20180304,0.872897,0.866393,0.877648,0.887795,0.893470,0.898150,0.895769,0.897469,0.892640,0.897588,0.891732,0.887982
20180316-20180328,0.860538,0.871516,0.882705,0.881906,0.880684,0.881604,0.885256,0.873128,0.872873,0.866187,0.878314,0.875640
20180409-20180421,0.948460,0.929908,0.912713,0.887666,0.867564,0.848735,0.829706,0.815956,0.800959,0.774731,0.847517,0.861640
20180503-20180515,0.918106,0.927019,0.935776,0.944261,0.947002,0.950492,0.956583,0.957870,0.958020,0.959103,0.950305,0.945423
20180527-20180608,0.901515,0.877216,0.848889,0.824443,0.793161,0.775270,0.749389,0.728383,0.715894,0.696553,0.771759,0.791071
20180620-20180702,0.949689,0.927372,0.908587,0.904888,0.900999,0.893989,0.895657,0.901213,0.907414,0.920500,0.903362,0.911031
20180807-20180819,0.871571,0.850294,0.826432,0.814842,0.795646,0.775330,0.775031,0.766127,0.753736,0.749635,0.784843,0.797865
20180831-20180912,0.863861,0.834112,0.816344,0.802380,0.787522,0.765809,0.753108,0.728578,0.711070,0.691920,0.762318,0.775470


<a id='transient_conclusion2'></a>
## Conclusion

Compared with percentage of total passed pairs, the mean value of percentage of passed pairs in all bin is a better indicator since it gives all bins same weight. 

In [60]:
percentage = np.count_nonzero(ratio_pd['mean']>thresthod)/n_ifgs

In [61]:
print(f"Percentage of interferograms passes the requirement (70%): {percentage}.")
if percentage >= 0.70:
    print('The interferogram stack passes the requirement.')
else:
    print('The interferogram stack fails the requirement.')

Percentage of interferograms passes the requirement (70%): 1.0.
The interferogram stack passes the requirement.


<div class="alert alert-warning">
Approach 2 final result for CentralValleyA144: 100% of interferograms passes the requirement.
</div>

<a id='transient_appendix'></a>
# Appendix: GNSS Position Plots

The relative position in LOS direction for all GNSS stations are plotted here.

In [62]:
gnss_time_series[1]

,BEPK,CACO,CAFP,CAHA,CAKC,CAND,CARH,CAWO,CRCN,DONO,...,P573,P789,P790,PKDB,POMM,RAPT,RNCH,TBLP,TRAN,WLHG
0,144.047348,878.855957,-423.828094,-155.689987,527.167053,-270.987854,-827.650696,-129.046005,-876.553650,191.638351,...,388.009369,629.913696,-269.708466,-192.827225,-157.080383,-433.926056,-601.373779,75.520126,-741.277710,-421.895569
1,141.318710,875.575317,-420.468658,-158.617172,526.022583,-274.104370,-827.547363,-131.713348,-880.717102,183.628738,...,385.767517,629.315796,-266.253601,-187.512573,-159.519485,-439.106171,-602.123657,74.819359,-744.281433,-424.141876
2,144.805511,878.123047,-422.783600,-158.909790,528.006897,-273.299225,-823.973999,-131.318436,-878.887329,189.378265,...,387.622131,632.161560,-263.294647,-192.931290,-157.491898,-438.707764,-602.302429,75.363487,-742.734192,-422.757629
3,146.757324,877.073975,-420.258484,-157.312897,527.452148,-274.138947,-827.697693,-129.911880,-876.706543,193.116791,...,387.320007,632.860413,-267.535095,-191.845291,-157.042679,-442.539124,-605.488953,77.122780,-740.115417,-423.223328
4,146.681320,877.319275,-423.840546,-155.731918,527.164001,-271.732727,-825.062012,-130.933868,-874.418640,192.470367,...,389.872070,632.412354,-262.350739,-192.193268,-155.493088,-450.452759,-598.615234,74.524284,-742.558838,-419.848389
5,141.011536,877.196228,-420.794281,-155.988312,524.441650,-274.559723,-828.775391,-134.378922,-879.648926,184.873169,...,386.527191,627.853760,-274.290619,-193.517929,-159.881027,-446.296326,-603.183777,72.662857,-744.626770,-423.519318
6,143.255386,876.617126,-424.181885,-157.020752,525.460327,-274.191010,-826.154480,-132.101715,-878.623962,186.111267,...,386.431122,628.515137,-266.018402,-191.262955,-159.706161,-413.688263,-603.831116,71.355194,-745.463135,-426.002136
7,146.116531,883.023376,-420.216034,-156.729553,528.290222,-272.544952,-825.648132,-128.112686,-882.505798,187.875809,...,391.821869,628.810974,-266.772034,-191.850189,-156.855011,-411.823395,-600.950134,76.156738,-742.942200,-421.783813
8,144.961472,877.932434,-419.608307,-158.756699,528.633423,-272.508606,-825.118225,-132.484222,-880.165833,191.694733,...,389.640106,630.828064,-270.253723,-189.531204,-155.902115,-411.023926,-601.050171,78.367889,-740.748108,-421.044769
9,147.580719,878.413635,-425.129883,-156.167831,527.586487,-271.862274,-826.320435,-130.966263,-881.060730,189.913696,...,388.930389,630.443665,-263.755432,-190.212662,-154.906891,-409.658142,-602.087830,75.519684,-740.097412,-418.087646


In [ ]:
for i in range(len(gnss_time_series)):
    start_time_str = ifgs_date[i,0].strftime('%Y%m%d')
    end_time_str = ifgs_date[i,1].strftime('%Y%m%d')
    for stn in gnss_time_series[i].columns:
        print(f'Plotting GPS postion from {start_time_str} to {end_time_str} at station: {stn}')
        series = gnss_time_series[i,str(stn)]
        plt.figure(figsize=(15,5))
        plt.title(f"station name: {stn}")
        plt.scatter(pd.date_range(start=ifgs_date[i,0], end=ifgs_date[i,1]),series)
        plt.xlabel('Time')
        plt.ylabel('Relative position in LOS direction (mm)')
        plt.savefig(work_dir/f'{start_time_str}_{end_time_str}_{stn}.jpg')
        plt.close()

Plotting GPS postion from 20180103 to 20180115 at station: BEPK
Plotting GPS postion from 20180103 to 20180115 at station: CACO
Plotting GPS postion from 20180103 to 20180115 at station: CAFP
Plotting GPS postion from 20180103 to 20180115 at station: CAHA
Plotting GPS postion from 20180103 to 20180115 at station: CAKC
Plotting GPS postion from 20180103 to 20180115 at station: CAND
Plotting GPS postion from 20180103 to 20180115 at station: CARH
Plotting GPS postion from 20180103 to 20180115 at station: GR8R
Plotting GPS postion from 20180103 to 20180115 at station: HOGS
Plotting GPS postion from 20180103 to 20180115 at station: HUNT
Plotting GPS postion from 20180103 to 20180115 at station: LAND
Plotting GPS postion from 20180103 to 20180115 at station: LOWS
Plotting GPS postion from 20180103 to 20180115 at station: MASW
Plotting GPS postion from 20180103 to 20180115 at station: MIDA
Plotting GPS postion from 20180103 to 20180115 at station: MNMC
Plotting GPS postion from 20180103 to 20